# <span style='color:#ff5f27'> 👨🏻‍🏫 GCS Online External Feature Group Creation</span>

Follow this [guide](https://docs.hopsworks.ai/3.1/user_guides/fs/storage_connector/creation/gcs/) to set up a connection to GCS.

In addition, you can read about [External Feature Groups](https://docs.hopsworks.ai/3.0/user_guides/fs/feature_group/create_external/).

## <span style='color:#ff5f27'> 🔮 Retrieve a Connector

Firstly, connect to feature store and then retrieve your **GCS storage connector**.

Replace `my_storage_connector_name` with your GCS storage connector name.

In [ ]:
import hsfs

# Connect to the Hopsworks feature store
hsfs_connection = hsfs.connection()

# Retrieve the metadata handle
feature_store = hsfs_connection.get_feature_store()

# Retrieve storage connector
connector = feature_store.get_storage_connector('my_storage_connector_name')

## <span style='color:#ff5f27'> 📝 Read Data </span>

You can retrieve your data by passing the next parameters of the `.read()` method:

- `data_format` - parquet, csv, orc, hudi, delta.
- `path` - the full object path where your table is located.

> ⚠️ **Don't** include dataset name in your path.

The `path` parameter should be in the next format:

`gs://{your_bucket_name}/{your_folder_name}/`

Replace `your_bucket_name` and `your_folder_name` with your GCS bucket name and folder name respectively.

In [ ]:
# read data into dataframe using path 
df = connector.read(
    data_format='parquet',
    path='gs://{your_bucket_name}/{your_folder_name}/'
)   

> **.read()** method returns a PySpark DataFrame.

In [ ]:
type(df)

In [ ]:
df.show(5)

## <span style='color:#ff5f27'> ⚙️ Feature Group Creation</span>

To create an **Online External Feature Group**, use `feature_store.create_external_feature_group()` method.

Provide the next parameters:
- `data_format` - data format of your table (parquet, csv, orc, hudi, delta).
- `path` - provide a folder name where your dataset is located in the next format: `{my_folder_name}/`.
- `storage_connector` - your **GCS Storage Connector**.
- `online_enabled` - set to `True`.

In [ ]:
my_external_fg = feature_store.create_external_feature_group(
    name="name_online_external_fg",
    version=1,
    description="Description of the Online External GCS Table",
    primary_key=['pk1_name','pk2_name'],
    event_time='event_time_col_name',
    data_format='{my_data_format}',
    path='{my_folder_name}/',
    storage_connector=connector,
    online_enabled=True,
)

Don't forget to register a feature group metadata object in Hopsworks!

In [ ]:
my_external_fg.save()

Now, you should be able to see your Online External Feature Group for the GCS table in the Hopsworks UI.

![GCS_table](images/online_external_fg.png)

---

## <span style='color:#ff5f27'> ⬇️ Insert to online storage</span>

You can enable online storage for external feature groups, however, the sync from the external storage to Hopsworks online storage needs to be done manually, so you need to acomplish the next steps:
- Filter data to sync to online.
- Read from external storage.
- Insert to online storage.

Replace `column_name` and `filter_value` with required **column name** and a **value** by which you will filter.

> ⚠️ Don't forget that all column names are set to lower case!

In [ ]:
column_name = 'column_name'
filter_value = 'filter_value'

In [ ]:
df = my_external_fg.filter(my_external_fg[column_name] == filter_value).read()

df.show(1)

Insert filtered data to online storage in order to be able to call `.get_feature_vector()` method.

In [ ]:
my_external_fg.insert(df, wait=True)

## <span style='color:#ff5f27'> ⚙️ Feature View Creation</span>

In [ ]:
query = my_external_fg.select_all()

In [ ]:
feature_view = feature_store.get_or_create_feature_view(
    name='my_feature_view_name',
    version=1,
    query=query
)

In [ ]:
vector = feature_view.get_feature_vector(
    entry = {"pk1_name": 1, "pk2_name": 2}
)
vector

---